In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
import category_encoders as ce
from sklearn.preprocessing import StandardScaler

In [6]:
LOCAL = '../data/tanzania/'
train_features = pd.read_csv(LOCAL+'train_features.csv')
train_labels = pd.read_csv(LOCAL+'train_labels.csv')
test_features = pd.read_csv(LOCAL+'test_features.csv')

In [14]:
X_train = train_features
y_train = train_labels['status_group']

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,test_size = 0.241717, random_state=42, stratify=y_train)

In [16]:
X_val.shape

(14358, 40)

In [18]:
assert X_val.shape == (14358, 40)

In [4]:
X_train.describe(exclude='number').T.sort_values(by='unique')

,count,unique,top,freq
recorded_by,44550,1,GeoData Consultants Ltd,44550
public_meeting,42082,2,True,38275
permit,42257,2,True,29099
source_class,44550,3,groundwater,34365
management_group,44550,5,user-group,39428
quantity_group,44550,5,enough,24909
quantity,44550,5,enough,24909
waterpoint_type_group,44550,6,communal standpipe,25910
quality_group,44550,6,good,38044
payment_type,44550,7,never pay,19035


In [5]:
categorical_features = ['waterpoint_type_group']
numeric_features = X_train.select_dtypes('number').columns.drop('id').tolist()
features = categorical_features + numeric_features

In [19]:
train_features.isna().sum()

id                           0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

In [6]:
X_train_subset = X_train[features]
X_val_subset = X_val[features]

scaler = StandardScaler()
X_train_subset[numeric_features] = scaler.fit_transform(X_train_subset[numeric_features])
X_val_subset[numeric_features] = scaler.transform(X_val_subset[numeric_features])

encoder = ce.OneHotEncoder(use_cat_names = True)
X_train_encoded = encoder.fit_transform(X_train_subset)
X_val_encoded = encoder.transform(X_val_subset)

logr = LogisticRegression(solver = 'lbfgs', multi_class = 'auto', max_iter = 10000)
logr.fit(X_train_encoded,y_train)
print('Validation Accuracy',logr.score(X_val_encoded,y_val))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

Validation Accuracy 0.6142760942760943


In [11]:
X_val_encoded.shape

(14850, 15)

In [7]:
y_pred = logr.predict(X_val_encoded)
pred_df = pd.DataFrame({'status_group':y_pred})

In [8]:
pred_df['id']=pred_df.index

In [12]:
pred_df.shape

(14850, 2)

In [10]:
pred_df.to_csv('Predictions',index = False)